In [1]:
import geopandas as gpd
import fiona
kml_file = "../files/CTO_113-11.kml"
kml_clean_file = "../files/Legend.kml"
import pandas as pd

gpd.io.file.fiona.drvsupport.supported_drivers['KML'] = 'rw'


In [5]:
df = pd.DataFrame()
# If we wanted to get all the folders in the kml we use this.
for layer in fiona.listlayers(kml_file):
    s = gpd.read_file(kml_file, driver='KML', layer=layer)
    df = pd.concat([df, s], ignore_index=True)

In [6]:
# However, at this point we only want the folder with the utility poles.
df = gpd.read_file(kml_clean_file, driver='KML')

""" Since the type of the column "geometry" is Shapely.Point
    we have to change it manually."""
from shapely import wkt
df['str_geometry'] = df.geometry.apply(lambda x: wkt.dumps(x))
df['str_geometry'] = df.str_geometry.apply(lambda x: x[9:].split()[:2])
df['str_geometry'] = df.str_geometry.apply(lambda x: [x[1],x[0]])
df['latitude'] = df.str_geometry.apply(lambda x: x[0])
df['longitude'] = df.str_geometry.apply(lambda x: x[1])
df['ini'] = 2


In [7]:
""" It'd be a great idea to get the distances between every single utility pole
    before starting the actual algorithm"""
import geopandas as gpd
from geopy.distance import geodesic
from scipy.spatial.distance import pdist, squareform
coords = df[['latitude', 'longitude']].values
dist_matrix = squareform(pdist(coords, lambda u, v: geodesic(u, v).meters))


In [ ]:
""" Now that we have a matrix, which each element Aij, represents the distance
    from pole i to pole j, we want to associate those distances with each pole
    """